# Before runing the notebook make sure u have aws configuration
>- Edit Access key ID
- Edit Secret access key


In [1]:
import boto3
import botocore
import paramiko

# Initialize network
---
# Create dynamodb database

In [7]:

# KEYS = [('user_id','HASH','S'), ('user_email','RANGE','S')] # (COL_NAME, KEY_TYPE, ATTR_TYPE)

KEYS = [('user_email','HASH','S')] # (COL_NAME, KEY_TYPE, ATTR_TYPE)
TABLE_NAME = 'users'
RW_CAPACITY = 10

def create_nodes_db(table_name):
    dynamodb = boto3.resource('dynamodb')
    try:
        table = dynamodb.create_table(
                    TableName=table_name,
                    KeySchema=[{'AttributeName':k[0],
                                'KeyType':k[1]} for k in KEYS],

                    AttributeDefinitions=[{'AttributeName':k[0],
                                           'AttributeType':k[2]} for k in KEYS],

                    ProvisionedThroughput={
                        'ReadCapacityUnits': RW_CAPACITY,
                        'WriteCapacityUnits': RW_CAPACITY})


        table.meta.client.get_waiter('table_exists').wait(TableName=TABLE_NAME)
        return table
    except dynamodb.meta.client.exceptions.ResourceInUseException:
        print(f'{TABLE_NAME} DB Already exists')
        return dynamodb.Table(TABLE_NAME)
     
table = create_nodes_db(TABLE_NAME)

users DB Already exists


---

# Create users

In [8]:
from user_controller import User, create_user

In [9]:


users = [User(user_email='ricky@gmail.com', first_name='ricky', last_name='dan', amount=1000, password='1234567'),
        User(user_email='dorel@gmail.com', first_name='dorel', last_name='sho', amount=1000, password='1234567'), 
        User(user_email='itay@gmail.com', first_name='itay', last_name='zei', amount=1000, password='1234567')]

for user in users:
    create_user(user)

# Get All users

In [10]:
from user_controller import get_all_users

users = get_all_users()
users

[user_email:dorel@gmail.com, first_name:dorel, last_name:sho, password:1234567, ammount:1000,
 user_email:ricky@gmail.com, first_name:ricky, last_name:dan, password:1234567, ammount:1000,
 user_email:itay@gmail.com, first_name:itay, last_name:zei, password:1234567, ammount:1000]

# Get users by key

In [11]:
from user_controller import get_users_by_key

key = 'ricky@gmail.com'
get_users_by_key(key=key)

user_email:ricky@gmail.com, first_name:ricky, last_name:dan, password:1234567, ammount:1000

# Update user

In [12]:
from user_controller import update_user

key = 'itay@gmail.com'
u_user = get_users_by_key(key)
u_user.amount = 3000
update_user(u_user)  

user_email:itay@gmail.com, first_name:itay, last_name:zei, password:1234567, ammount:3000

# Delete User by key

In [13]:
from user_controller import delete_user

key = 'ricky@gmail.com'
delete_user(key)
get_all_users()

[user_email:dorel@gmail.com, first_name:dorel, last_name:sho, password:1234567, ammount:1000,
 user_email:itay@gmail.com, first_name:itay, last_name:zei, password:1234567, ammount:3000]

# Transacations

# Create Transcation DB

In [26]:
KEYS = [('transaction_id','HASH','S')] # (COL_NAME, KEY_TYPE, ATTR_TYPE)
TABLE_NAME = 'transactions'
RW_CAPACITY = 10

table = create_nodes_db(TABLE_NAME)

transactions DB Already exists


# Create Transaction

In [15]:
from transaction_controller import create_transaction, Transaction

In [16]:
create_transaction(Transaction(src='dorel@gmail.com', dst='ricky@gmail.com', amount=40));

# Get Transaction by key

In [ ]:
from transaction_controller import get_transaction_by_key


get_transaction_by_key(key='2808343791251813472')

# Get Transactions by dst

In [28]:
from transaction_controller import get_transactions_by_dst


In [30]:
for e in get_transactions_by_dst('itay@gmail.com'):
    print(e.__dict__)

{'transaction_id': '-7179700992320699511', 'src': 'dorel@gmail.com', 'dst': 'itay@gmail.com', 'amount': 20, 'signed': False}


# Update Transaction

In [20]:
from transaction_controller import update_transaction

other = Transaction(transaction_id='2808343791251813472',src=None, dst=None, amount=None, signed=True)

In [ ]:
update_transaction(other)

In [ ]:
get_transaction_by_key('2808343791251813472')

# Delete Transaction


In [ ]:
from transaction_controller import delete_transaction

In [ ]:
delete_transaction('2808343791251813472')

In [ ]:
for e in get_transactions_by_dst('ricky@gmail.com'):
    print(e.__dict__)

# Make Transaction 
- ricky->dorel with amount 20

In [22]:
get_all_users()

[user_email:dorel@gmail.com, first_name:dorel, last_name:sho, password:1234567, ammount:1000,
 user_email:itay@gmail.com, first_name:itay, last_name:zei, password:1234567, ammount:3000]

In [24]:
amount=20
ricky = get_users_by_key(key='dorel@gmail.com')
dorel = get_users_by_key(key='itay@gmail.com')

In [25]:
new_tr = Transaction(src=ricky.user_email, dst=dorel.user_email, amount=20)
create_transaction(new_tr);

# Sign transactio
dorel+20->ricky-20

In [ ]:
get_transactions_by_dst(dorel.user_email)

In [ ]:

other = Transaction(transaction_id='3538720164617471173',src=None, dst=None, amount=20, signed=True)
u_ricky = get_users_by_key(key='ricky@gmail.com')
u_dorel = get_users_by_key(key='dorel@gmail.com')
u_ricky.amount-=20
u_dorel.amount+=20
update_user(u_ricky)
update_user(u_dorel)
update_transaction(other)
# send signed tio master

In [ ]:
get_all_users()

In [ ]:
u_dorel

----------

# Create buckets 

- **blockchain-utils** - files to run on launch
    - upload the file init files to the buckets
    - NOTE: Maybe image is all we need
- **blocks** - where the blocks will be stored

In [ ]:
def create_bucket(bucket_name, region='eu-central-1'):
    # Create bucket
    try:
        s3_client = boto3.client('s3', region_name=region)
        location = {'LocationConstraint': region}
        s3_client.create_bucket(Bucket=bucket_name,
                                CreateBucketConfiguration=location)
    except ClientError as e:
        print(e)
        return False
    return True

# create_bucket('blockchain-utils')
create_bucket('ds-blocks')

# Create SQS 
- Ledger - where the signed transcation will be stored

In [ ]:
def create_ledger_sqs(name='ledger'):
    sqs = boto3.resource('sqs')
    
    private_sqs_params = dict(
      QueueName=f'{name}.fifo',
      Attributes={
          'FifoQueue': 'True'
      },
    )
    return sqs.create_queue(**private_sqs_params)

ledger = create_ledger_sqs()
ledger.url

# Create N user Instances
- crteate an image including python3 and boto3 


In [ ]:
from botocore.exceptions import ClientError
N = 4
IMAGE_ID = 'ami-07e0b9cd44928fe2f' # Amazon Linux 2 AMI (HVM), SSD Volume Type (My AMI)
KEY_NAME = 'DSKey' # edit to your name of <key-name>.pem file
SECURITY_GROUP = ['launch-wizard-6']

In [ ]:
ec2 = boto3.resource('ec2')
response = ec2.create_instances(
            ImageId=IMAGE_ID,
            MinCount=1, # [Optional] Minimum instances to launch
            MaxCount=1, # [Optional] Maximum instances to launch
            InstanceType='t2.micro',
            KeyName=KEY_NAME,
            SecurityGroups=SECURITY_GROUP
        )


In [ ]:
response[0].wait_until_running

In [ ]:

def create_instance(n_count=N):
    ec2 = boto3.resource('ec2')

    try:
        ec2.create_instances(
            ImageId=IMAGE_ID,
            MinCount=n_count, # [Optional] Minimum instances to launch
            MaxCount=n_count, # [Optional] Maximum instances to launch
            InstanceType='t2.micro',
            KeyName=KEY_NAME,
            SecurityGroups=SECURITY_GROUP
        )
        print(f'ec2 Created Successfully')
    except ClientError as e:
        print(e)
        print(f'\nFailed to create instance')
        return False
    return True

create_instance(n_count=1)

In [ ]:
def get_instances():
    instances = []
    ec2 = boto3.client('ec2')
    response = ec2.describe_instances()

    for res in response['Reservations']:
        for inst in res['Instances']:
            instances.append({'DNS': inst['PublicDnsName'],
                              'Adress': inst['PublicIpAddress'],
                              'ID': str(sum([int(x) for x in inst['PublicIpAddress'].split('.')]))
                              })
    return instances

def find_higher_ranks(nodes, my_rank):
    rv = []
    for node in nodes:
        if int(node['ID']) > int(my_rank):
            rv.append(node)

    return rv

nodes = get_instances()
for node in nodes:
    print(node['ID'])

find_higher_ranks(nodes,0)    

In [ ]:
ec2 = boto3.client('ec2')
response = ec2.describe_instances()
response['Reservations']

# Send CMD to instance to run


In [ ]:
PATH_TO_KEY = '../DSKey.pem'

def send_cmd_to_instance(instance_public_dns, cmd):
    key = paramiko.RSAKey.from_private_key_file(PATH_TO_KEY)
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

    try:
        client.connect(hostname=instance_public_dns, username="ec2-user", pkey=key)
        stdin, stdout, stderr = client.exec_command(cmd)
        print(stdout.read().decode('utf-8'))
        client.close()
    except Exception as e:
        print(e)

for inst in get_instances():
    if inst != '':
        print(inst)
        send_cmd_to_instance(instance_public_dns=inst,
                             cmd='python3 init_user_instance.py')       


# Init Instance test
assumptions:
 - DB: Table 'node' were created


In [ ]:
import time

myMessages = None
# Dummy funcs
def get_instance_ID():
    return 2
def get_public_DNS():
    return '127.0.0.2'

def create_sqs():
    private_sqs_params = dict(
      QueueName=f'{get_instance_ID()}.fifo',
      Attributes={
          'FifoQueue': 'True'
      },
    )
    
    # Get the service resource
    sqs = boto3.resource('sqs')
    return sqs.create_queue(**private_sqs_params)
    
def init_instance(table = 'nodes'):
#     myMessages = create_sqs()
    dynamodb = boto3.resource('dynamodb')
    while True:
        try:
            
            table = dynamodb.Table(table)
            break
        except:
            time.sleep(5)
            continue
            
    
    response = table.put_item(
           Item={
                'node_id': str(get_instance_ID()),
                'node_dns': get_public_DNS(),
                'message_q': f'{get_instance_ID()}.fifo',
                'role': 'user',
            }
        )
    
    print('Node initialized')


# Suggestions:

In [ ]:
def print_menu():
    print(f'1) make transaction:')

print_menu()   
val=''
while val != 'close':
    val = input("Enter your value: ") 
    print(val)


- Flask